In [ ]:
import os
import urllib.request
from flask import Flask, flash, request, redirect, render_template
from werkzeug.utils import secure_filename
import pandas as pd
import numpy as np
import time
from collections import Counter
import csv
#from playsound import playsound
import pickle
from flask_cors import CORS, cross_origin
from flask import url_for

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'

UPLOAD_FOLDER = 'C:/Users/zainab/upload'
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
scores = []
labels = []

def predictTheValue(file):
    features_snps = []

    with open('features_with_importance.csv', newline='') as csvfile:
        features = list(csv.reader(csvfile))
        features = features[1:]
        for feature in features:
            features_snps.append(feature[0])
        features_snps = set(features_snps)

    # construct user dataframe from user's 23andme file
    print("Opening file: {} ...".format(file))
    user_df = pd.read_csv(file, delim_whitespace=True, names=['rsid', 'chromosome', 'position', 'genotype'])
    user_df = user_df[['rsid','genotype']]
    user_df = user_df.set_index(['rsid'])
    user_df = user_df[user_df.index.isin(features_snps)] 
    user_df = user_df.transpose()
    #print(user_df)

    # Create dataframe of alleles count 
    alleles_count_df = pd.read_csv('alleles_count.csv', delim_whitespace=True, names=['rsid', 'genotype1', 'count1', 'genotype2', 'count2', 'genotype3', 'count3'])
    alleles_count_df = alleles_count_df[['rsid', 'genotype1', 'count1','genotype2', 'count2']]
    alleles_count_df = alleles_count_df.set_index(['rsid'])
    alleles_count_df = alleles_count_df[alleles_count_df.index.isin(features_snps)]
    alleles_count_df = alleles_count_df.reset_index()
    #print(alleles_count_df)

    # Encoding
    for (columnName, columnData) in user_df.iteritems():

        if columnName in alleles_count_df.rsid.tolist():
            major_allele = alleles_count_df.loc[alleles_count_df.rsid == columnName,'genotype1'].tolist()[0]
            user_genotype = user_df.at['genotype', columnName]

            if user_genotype == "AG" or user_genotype == "GA" or user_genotype == "CT" or user_genotype == "TC": 
                user_df.at['genotype', columnName] = 2
            elif major_allele == "AG" or major_allele == "GA" or major_allele == "CT" or major_allele == "TC":
                second_major_allele = alleles_count_df.loc[alleles_count_df.rsid == columnName,'genotype2'].tolist()[0]
                if user_genotype == second_major_allele:
                    user_df.at['genotype', columnName] = 1
                else:
                    user_df.at['genotype', columnName] = 3
            else:
                if user_genotype == major_allele:
                    user_df.at['genotype', columnName] = 1
                else:
                    user_df.at['genotype', columnName] = 3
            
    snp = user_df.columns.values
    X = user_df[snp].values
    #print(X)
    # Load model
    with open('pickle_model.pkl', 'rb') as file:
        pickle_model = pickle.load(file)
    # Predict
    Ypredict = pickle_model.predict_proba(X)
    print(Ypredict)
    print(pickle_model.classes_)
    scores=pickle_model.predict_proba(X)
    labels=pickle_model.classes_
    data= ' '.join(map(str, scores[0])) 
    data=data+" "+listToString(labels)
    flash(data)

def listToString(s):  
    
    # initialize an empty string 
    str1 = " " 
    
    # return string   
    return (str1.join(s)) 

@app.route('/')
def upload_form():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        f = request.files['file']
        predictTheValue(f)
        #to save file
        #filename = secure_filename(f.filename)
        #f.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        return redirect('/')

if __name__ == '__main__':
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Apr/2020 22:37:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2020 22:37:43] "GET /static/images/icons/favicon.ico HTTP/1.1" 200 -


Opening file: <FileStorage: '682.23andme.317' ('application/octet-stream')> ...


C:\Users\zainab\anaconda3\lib\site-packages\flask\app.py:1950: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  rv = self.dispatch_request()
C:\Users\zainab\anaconda3\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator SVC from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
127.0.0.1 - - [23/Apr/2020 22:37:54] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [23/Apr/2020 22:37:54] "GET / HTTP/1.1" 200 -


[[0.08902245 0.25685608 0.65412147]]
['blue' 'brown' 'green']
